In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
path = os.getcwd()
parent = os.path.dirname(path)
os.chdir(parent + '/dataset_cleaned/')

In [9]:
monthly_generation = pd.read_csv('monthly_generation.csv')

In [10]:
monthly_generation

,Year,Plant Id,State,Aggregated Fuel Group,Month,Generation
0,2013,3,AL,COAL,Apr,244468.68
1,2013,3,AL,COAL,Aug,506485.96
2,2013,3,AL,COAL,Dec,219258.08
3,2013,3,AL,COAL,Feb,334765.21
4,2013,3,AL,COAL,Jan,545082.12
...,...,...,...,...,...,...
243607,2020,99999,WI,COAL,Mar,0.00
243608,2020,99999,WI,COAL,May,0.00
243609,2020,99999,WI,COAL,Nov,0.00
243610,2020,99999,WI,COAL,Oct,0.00


### Calculate the coal percentage matrix

In [13]:
generation = pd.read_csv('coal_monthly_generation_by_state.csv')

In [20]:
# calculate month / year percentage for each state

a = pd.pivot_table(monthly_generation, index=['Year', 'State'], columns=['Month'], values='Generation', dropna=False)
a = a[['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug','Sep', 'Oct', 'Nov', 'Dec']]

b = monthly_generation.groupby(['Year', 'State']).aggregate({'Generation': 'sum'})

generation_monthly_percentage = np.divide(a, b)
generation_monthly_percentage.head()

/var/folders/xy/8mx9wl7x3h19xrmkncxcmw3r0000gn/T/ipykernel_14595/10146522.py:8: FutureWarning: Calling a ufunc on non-aligned DataFrames (or DataFrame/Series combination). Currently, the indices are ignored and the result takes the index/columns of the first DataFrame. In the future , the DataFrames/Series will be aligned before applying the ufunc.
Convert one of the arguments to a NumPy array (eg 'ufunc(df1, np.asarray(df2)') to keep the current behaviour, or align manually (eg 'df1, df2 = df1.align(df2)') before passing to the ufunc to obtain the future behaviour and silence this warning.
  generation_monthly_percentage = np.divide(a, b)


Month            Jan       Feb       Mar       Apr       May       Jun  \
Year State                                                               
2013 AK     0.004998  0.004144  0.004478  0.003835  0.003873  0.003649   
     AL     0.001346  0.001389  0.001484  0.001260  0.001279  0.001539   
     AR     0.002669  0.002431  0.002485  0.002014  0.002111  0.002812   
     AZ     0.001889  0.001426  0.001610  0.001877  0.001929  0.002592   
     CA     0.000245  0.000226  0.000216  0.000186  0.000196  0.000232   

Month            Jul       Aug       Sep       Oct       Nov       Dec  
Year State                                                              
2013 AK     0.004088  0.003937  0.004103  0.003649  0.004239  0.005007  
     AL     0.001560  0.001704  0.001494  0.001434  0.001367  0.001386  
     AR     0.003026  0.003044  0.002498  0.001995  0.001718  0.002607  
     AZ     0.003099  0.003110  0.002664  0.002060  0.001790  0.002270  
     CA     0.000294  0.000289  0.000289  0.000246  0.000245  0.000284

In [49]:
yearly_generation_emission = pd.read_csv('yearly_generation_emission.csv').set_index(['Year', 'State', 'Plant Code'])

In [51]:
# calculate the monthly generation according to the state's monthly generation percentage

data = []
for p in yearly_generation_emission.index: 

    year, state, plant = p[0], p[1], p[2]

    generation = yearly_generation_emission.loc[p]['Generation']

    percentage = generation_monthly_percentage.loc[(year, state)]

    print(generation.shape)
    data.append([year, state, plant] + list(np.array(generation) * np.array(percentage)))

monthly = pd.DataFrame(data, columns=['Year', 'State', 'Plant Code', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug',
                  'Sep', 'Oct', 'Nov', 'Dec'])

monthly = pd.melt(monthly, id_vars = ['Year', 'State', 'Plant Code']).rename({'variable': 'Month', 'value': 'Generation'}, axis=1)

(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(2,)


ValueError: operands could not be broadcast together with shapes (2,) (12,) 

In [43]:
monthly

,Year,State,Plant Code,Month,Generation
0,2013,AK,79,Jan,1.775186e+07
1,2013,AK,6288,Jan,1.805624e+07
2,2013,AK,50308,Jan,1.276316e+07
3,2013,AK,50392,Jan,6.748508e+06
4,2013,AK,50711,Jan,3.709194e+06
...,...,...,...,...,...
42787,2020,WY,55479,Dec,6.232713e+07
42788,2020,WY,56319,Dec,6.390252e+07
42789,2020,WY,56596,Dec,7.521967e+07
42790,2020,WY,56609,Dec,2.862475e+08


In [44]:
# get the generation to emission function

model = smf.ols(formula='Emission ~ Generation', data=generation_emission)
model = model.fit()

In [45]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               Emission   R-squared:                       0.992
Model:                            OLS   Adj. R-squared:                  0.992
Method:                 Least Squares   F-statistic:                 4.207e+05
Date:                Mon, 24 Oct 2022   Prob (F-statistic):               0.00
Time:                        21:31:11   Log-Likelihood:                -50722.
No. Observations:                3566   AIC:                         1.014e+05
Df Residuals:                    3564   BIC:                         1.015e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   1.649e+05   7599.864     21.698      0.000     1.5e+05     1.8e+05
Generation     0.0011   1.64e-06    648.622      0.000       0.001       0.001
==============================================================================
Omnibus:                     1874.382   Durbin-Watson:                   1.682
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            37282.769
Skew:                           2.052   Prob(JB):                         0.00
Kurtosis:                      18.300   Cond. No.                     5.76e+09
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.76e+09. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [55]:
# use the generation to emission function to get the monthly output for each power plant
monthly['Emission (Predicted)'] = model.predict(monthly)

In [62]:
monthly.to_csv('predicted_monthly_emission.csv', index=False)